# Path analysis for data discovery

In [105]:
from setup import setup
import sys
from pathlib import Path
import pandas as pd
import json
from typing import Tuple, Type

sys.path.append(f"{Path.cwd().parent.absolute()}/")

db = setup()


## Setting parameters

In [106]:
OID = 139
skip_features = []
datetime_format = ""
# "其他", "高中高職", "專科", "大學", "碩士", "博士"
concept_hierarchy = {
    "教育程度類別": {
        "hierarchy": {"高中以下": ["高中高職", "其他"], "大專": ["專科", "大學"], "研究所": ["碩士", "博士"]},
        "order": ["高中以下", "大專", "研究所"],
    }
}
target = "信用卡交易金額[新台幣]"

## Fetch data from database

In [107]:
from sqlalchemy import text

query = text(f"SELECT * FROM [RawDB].[dbo].[D{OID}]")
data = db.execute(query)

query = text("SELECT * FROM [DV].[dbo].[Object] where OID = :OID")
object_table = db.execute(query, OID=OID).fetchall()

df = pd.DataFrame(data.fetchall())
copy_of_df = df.copy()

data_name = pd.DataFrame(object_table)["CName"][0]
print(f"Data name: {data_name}")

column_names = df.columns.tolist()


Data name: 各教育程度持卡人於十六縣消費樣態 (男女)


## Numerical column handler

In [108]:
quantile_mapping = {}

numerical_column_tuple: Tuple[str, int, int] = []
for col in column_names:
    column_ratio = len(df[col].unique()) / df[col].count()
    is_categorical_column = df[col].dtype == "object" or df[col].dtype == "category" or column_ratio < 0.01
    is_numerical_column = df[col].dtype == "int64" and not is_categorical_column
    if is_numerical_column:
        numerical_column_tuple.append([col, df[col].min(), df[col].max()])

numerical_column_tuple

quantile_numeric = []

for tuple in numerical_column_tuple:
    col = tuple[0]
    discrete_bin_num = 3
    quantile_labels = ["low", "middle", "high"]
    quantile = pd.qcut(df[col], q=[0, 0.1575, 0.8425, 1])
    quantile_numeric.append(col)
    df[col] = pd.qcut(df[col], q=[0, 0.1575, 0.8425, 1],
                      labels=quantile_labels)
    quantile_mapping[col] = pd.Series(
        data=quantile.unique().tolist(), index=df[col].unique().tolist())


## Datetime column handler

In [109]:
from datetime import datetime
from math import log
from pandas import DatetimeIndex


datetime_format_list = ["%Y-%m-%d", "%Y-%m", "%Y%m%d", "%Y%m", "%Y"]
datetime_column_tuple: Tuple[str, Type[pd.Timestamp], Type[pd.Timestamp]] = []

if len(datetime_format) != 0:
    datetime_format_list.insert(0, datetime_format)


# to_datetime reference: https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

for col in column_names:
    is_datetime = df[col].dtype == "datetime64[ns]"
    if is_datetime:
        datetime_column_tuple.append(
            [
                col,
                df[col].min(),
                df[col].max(),
            ]
        )
        continue
    for test_format in datetime_format_list:
        is_numeric = df[col].dtype == "int64"
        is_datetime = is_numeric and (
            True
            not in pd.to_datetime(arg=df[col].astype("str"), format=test_format, errors="coerce")
            .isna()
            .value_counts()
            .index.to_list()
        )
        if is_datetime:
            parsed_datetime = pd.to_datetime(arg=df[col], format=test_format, errors="coerce")
            df[col] = parsed_datetime
            datetime_column_tuple.append(
                [
                    col,
                    parsed_datetime.min(),
                    parsed_datetime.max(),
                ]
            )
            break

datetime_column_tuple

# Quantize datetime problem reference:
# https://stackoverflow.com/questions/43500894/pandas-pd-cut-binning-datetime-column-series
datetime_columns = []


def count_feature_gain(
    feature_name: str, target_name: str, feature_values: Type[pd.Series], target_values: Type[pd.Series]
):
    part_df = pd.DataFrame(columns=[feature_name, target_name])
    part_df[feature_name] = feature_values.astype("str")
    part_df[target_name] = target_values

    # Quantity of S
    total_quantity = len(part_df[target_name])
    class_quantity = len(part_df[target_name].unique())
    gain = 0

    for value in part_df[feature_name].dropna().unique().tolist():
        # Filter target attribute correspond the value of target
        value_df = part_df.loc[part_df[feature_name] == value]
        value_quantity = len(value_df[target_name])
        target_value_counts = value_df[target_name].value_counts().values.tolist()
        H_sv = 0
        for value_count in target_value_counts:
            if value_count != 0:
                H_sv += -(value_count / value_quantity) * log(value_count / value_quantity, class_quantity)
        gain += value_quantity / total_quantity * H_sv

    return gain


def fill_label(freq: str, datetime_manifest: Type[DatetimeIndex]):
    labels: list[str] = []

    match freq:
        case "year":
            labels = [str(year) for year in list(datetime_manifest.year)]
            if len(labels) >= 0:
                labels.pop()
        case "quarter":
            for i in range(1, len(datetime_manifest)):
                lower_bound = datetime_manifest[i - 1].month
                higher_bound = datetime_manifest[i].month

                match [lower_bound, higher_bound]:
                    case [1, 4]:
                        labels.append("Q1")
                    case [4, 7]:
                        labels.append("Q2")
                    case [7, 10]:
                        labels.append("Q3")
                    case [10, 1]:
                        labels.append("Q4")
                    case _:
                        label_mapping = pd.Series(data=["Q1", "Q2", "Q3", "Q4"], index=[1, 4, 7, 10])
                        labels.append(label_mapping[lower_bound])
        case "month":
            labels = [str(month) for month in list(datetime_manifest.month)]
            if len(labels) >= 0:
                labels.pop()
        case "week":
            labels = [
                "{}".format(
                    datetime.strftime(datetime_manifest[i], format="%U"),
                )
                for i in range(0, len(datetime_manifest))
            ]
            if len(labels) >= 0:
                labels.pop()
        case "day":
            labels = [str(day) for day in list(datetime_manifest.day)]
            if len(labels) >= 0:
                labels.pop()
        case _:
            pass

    return labels


for tuple in datetime_column_tuple:
    # date_range reference: https://pandas.pydata.org/docs/reference/api/pandas.date_range.html
    # Frequency reference: https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases

    col = tuple[0]
    datetime_columns.append(col)

    # * Compare which datetime frequency is the best
    datetime_freq_manifest = ["year", "quarter", "month", "week", "day"]
    freq_manifest = ["YS", "QS", "MS", "W", "D"]
    freq_mapping = pd.Series(data=freq_manifest, index=datetime_freq_manifest)
    datetime_freq_entropy = []

    for datetime_freq in datetime_freq_manifest:
        datetime_manifest: Type[pd.DatetimeIndex] = pd.date_range(
            start=tuple[1], end=tuple[2], freq=freq_mapping[datetime_freq]
        )
        datetime_manifest = datetime_manifest.union([tuple[2]])
        labels = fill_label(datetime_freq, datetime_manifest)
        discrete_datetime_series = pd.Series(
            pd.cut(df[col], bins=datetime_manifest, labels=labels, include_lowest=True, ordered=False)
        )

        feature_entropy = count_feature_gain(
            feature_name=col, target_name=target, feature_values=discrete_datetime_series, target_values=df[target]
        )
        datetime_freq_entropy.append(feature_entropy)

    difference_mean = 0

    for i in range(1, len(datetime_freq_entropy)):
        difference_mean += abs(datetime_freq_entropy[i] - datetime_freq_entropy[i - 1])

    difference_mean /= len(datetime_freq_entropy)

    best_index = datetime_freq_entropy.index(min(datetime_freq_entropy))
    # best_freq = datetime_freq_manifest[best_index] if difference_mean > 0.01 else "year"
    best_freq = datetime_freq_manifest[best_index]

    total_quantity = len(df[target])
    class_value_quantity = len(df[target].unique())
    H_s = [
        -(target_value / total_quantity) * log(target_value / total_quantity, class_value_quantity)
        for target_value in df[target].value_counts().values.tolist()
    ]
    entropy = sum(H_s)

    # ! test #####
    print(f"E(S) entropy: {entropy}")
    print(f"信用卡交易筆數 gain: {count_feature_gain('信用卡交易筆數', target, df['信用卡交易筆數'], df[target])}")
    print(datetime_freq_manifest)
    print(datetime_freq_entropy)
    print(best_freq)

    datetime_manifest: Type[pd.DatetimeIndex] = pd.date_range(
        start=tuple[1], end=tuple[2], freq=freq_mapping[best_freq]
    )
    datetime_manifest = datetime_manifest.union([tuple[2]])
    labels = fill_label(best_freq, datetime_manifest)

    quantile_interval = pd.cut(df[col], bins=datetime_manifest, include_lowest=True)
    quantile_mapping[col] = pd.Series(data=quantile_interval.tolist(), index=df[col].tolist())
    df[col] = pd.Series(pd.cut(df[col], bins=datetime_manifest, labels=labels, include_lowest=True, ordered=False))

E(S) entropy: 0.765864665210255
信用卡交易筆數 gain: 0.4366163559257483
['year', 'quarter', 'month', 'week', 'day']
[0.7648345266342315, 0.7654120571180669, 0.7649585157027823, 0.7647423802512571, 0.7654750774972149]
week


## Concept hierarchy handler

In [142]:
print(df["性別"].replace(to_replace=[1, 2], value="test"))

for col in concept_hierarchy:
    hierarchy = concept_hierarchy[col]["hierarchy"]
    order = concept_hierarchy[col]["order"]
    for concept in hierarchy:
        # df[col].replace(to_replace=)
        pass
    print(hierarchy)
    print(order)
    pass

0         test
1         test
2         test
3         test
4         test
          ... 
147835    test
147836    test
147837    test
147838    test
147839    test
Name: 性別, Length: 147840, dtype: object
{'高中以下': ['高中高職', '其他'], '大專': ['專科', '大學'], '研究所': ['碩士', '博士']}
['高中以下', '大專', '研究所']


## Target and features handler

In [ ]:
X: pd.DataFrame
try:
    X = df.drop([target] + skip_features, axis=1)
except KeyError:
    print("Column of target or skip features are not exist in data frame")

feature_names = X.columns.tolist()

y = df[target].astype("string")
class_names = y.unique().tolist()


## Encode category column

In [ ]:
import category_encoders as ce

category_frame = X.select_dtypes(include=["object", "category"])
encoder = ce.OrdinalEncoder(cols=category_frame.columns)
X = pd.DataFrame(encoder.fit_transform(X))

category_column_mapping = encoder.mapping
category_column_mapping


[{'col': '年月',
  'mapping': 43      1
  48      2
  52      3
  05      4
  09      5
  13      6
  17      7
  22      8
  26      9
  31     10
  35     11
  39     12
  44     13
  18     14
  53     15
  04     16
  08     17
  12     18
  21     19
  25     20
  30     21
  34     22
  47     23
  NaN    24
  dtype: int64,
  'data_type': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                    '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
                    '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
                    '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
                    '41', '42', '43', '44', '45', '46', '47', '48', '49', '50',
                    '51', '52', '53'],
  , ordered=False)},
 {'col': '地區',
  'mapping': 南投縣     1
  雲林縣     2
  宜蘭縣     3
  嘉義縣     4
  屏東縣     5
  嘉義市     6
  花蓮縣     7
  台東縣     8
  澎湖縣     9
  金門縣    10
  連江縣    11
  基隆市    12
  新竹市    13
  新竹

## Fitting decision tree model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

row_counts = len(X.index)
max_depth = 30
min_samples_split = 0
min_samples_leaf = 0

is_big_data = row_counts > 10000

if is_big_data:
    # 確保葉節點有足夠的樣本進行有意義的分析，同時避免過度細分
    # 100 - 1000
    min_samples_leaf = 100
    # 確保在分割內部節點之前有足夠的樣本數
    # 10 - 50
    min_samples_split = 10
else:
    # 確保每個葉節點至少有一些樣本進行分析
    # 1 or 2
    min_samples_leaf = 1
    # 確保在內部節點的樣本數較少時也可以進行分割
    # 2 - 5
    min_samples_split = 2


clf = DecisionTreeClassifier(
    criterion="entropy",
    max_depth=max_depth,
    random_state=42,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
)

clf.fit(X, y)


DecisionTreeClassifier(criterion='entropy', max_depth=30, min_samples_leaf=100,
                       min_samples_split=10, random_state=42)

### Feature importance

In [ ]:
feature_importance = clf.feature_importances_
feature_importance_pairs = list(zip(feature_names, feature_importance))
feature_importance_pairs.sort(key=lambda pair: pair[1], reverse=True)
feature_importance_pairs


[('信用卡交易筆數', 0.5646291567058018),
 ('地區', 0.17382797145410953),
 ('產業別', 0.11264911405292852),
 ('教育程度類別', 0.10971390412978371),
 ('性別', 0.03503453255923295),
 ('年月', 0.00414532109814348)]

### Cross validation and model accuracy

In [ ]:
from sklearn.model_selection import cross_val_score

# cv: k-fold, default is 5-fold
cross_validation_score = cross_val_score(clf, X, y, cv=5)
print("交叉驗證分數:", cross_validation_score)
print("平均分數:", cross_validation_score.mean())


交叉驗證分數: [0.87313988 0.85819129 0.88632982 0.88551813 0.85200216]
平均分數: 0.8710362554112553


## Resolve decision tree path data as json format

In [ ]:
from dataclasses import dataclass


# Type definition of decision tree path data


@dataclass
class DecisionTreeNode:
    id: int
    labels: list[str]


@dataclass
class DecisionTreeEdge:
    id: int
    label: str
    head: int
    tail: int


@dataclass
class DecisionTreeGraph:
    nodes: list[DecisionTreeNode]
    edges: dict[str, DecisionTreeEdge]  # node1_node2 as key value


@dataclass
class DecisionTreePath:
    path: list[int]
    nodeLabel: dict[int, list[str]]


In [ ]:
from sklearn import tree
from graphviz import Source

out_file_path = f"{Path.cwd().absolute()}/temp/temp.dot"

# * Scikit-learn decision tree:
# Using optimized version of the CART algorithm
# Not support categorical variable for now, that is, categorical variable need to encode

# * Entropy range:
# From 0 to 1 for binary classification (target has only two classes, true or false)
# From 0 to log base 2 k where k is the number of classes

class_names = clf.classes_.tolist()

dot_file = tree.export_graphviz(
    clf,
    out_file=out_file_path,
    feature_names=feature_names,
    class_names=class_names,
    max_depth=max_depth,
    label="all",
    rounded=True,
    filled=True,
)

with open(out_file_path, "r", encoding="utf-8") as f:
    dot_file = f.read()

# Use graphviz lib to convert dot format to json format
source = Source(dot_file)
json_graph = source.pipe(format="json").decode("utf-8")
dict_graph: dict = json.loads(json_graph)

# Filter needed part
nodes = list(
    map(
        lambda o: {"id": o.get("_gvid"), "labels": o.get(
            "label").split("\\n")},
        dict_graph.get("objects"),
    )
)

edges = dict(
    map(
        lambda o: (
            str(o.get("tail")) + "_" + str(o.get("head")),
            {
                "id": o.get("_gvid"),
                "label": o.get("headlabel"),
                "head": o.get("tail"),
                "tail": o.get("head"),
            },
        ),
        dict_graph.get("edges"),
    )
)


## Store useful information

In [ ]:
from math import nan

data_information: dict[str, str or list or dict] = {}
data_information["target_name"] = target
data_information["target_values"] = class_names
data_information["feature_names"] = feature_names

# Numeric
feature_values: dict[str, dict[str, str or list]] = {}

for n in numerical_column_tuple:
    feature_values[n[0]] = {"type": "numeric", "value": [n[1], n[2]]}

# Datetime
for part_df in datetime_column_tuple:
    format = "%Y-%m-%d %X"
    feature_values[part_df[0]] = {
        "type": "datetime",
        "value": [part_df[1].strftime(format), part_df[2].strftime(format)],
    }

# Category
for c in category_column_mapping:
    is_datetime_column = c["col"] in datetime_columns
    feature_values[c["col"]] = {
        "type": "datetime" if is_datetime_column else "category",
        "value": (c["mapping"].index.to_list()),
        "mapping": pd.Series(dict((v, k) for k, v in c["mapping"].items())),
    }
    feature_values[c["col"]]["value"].pop()

unstored_features = list(set(feature_names) - set(list(feature_values.keys())))

for f in unstored_features:
    split_value = df[f].unique().astype("str").tolist()
    df[f] = df[f].astype("str")

    mapping_pairs = dict((i + 1, split_value[i])
                         for i in range(len(split_value)))
    mapping_pairs[-2] = nan
    mapping = pd.Series(mapping_pairs)
    feature_values[f] = {
        "type": "category",
        "value": split_value,
        "mapping": mapping,
    }

data_information["feature_values"] = feature_values
data_information


{'target_name': '信用卡交易金額[新台幣]',
 'target_values': ['high', 'low', 'middle'],
 'feature_names': ['年月', '地區', '產業別', '性別', '教育程度類別', '信用卡交易筆數'],
 'feature_values': {'信用卡交易筆數': {'type': 'category',
   'value': ['low', 'middle', 'high'],
   'mapping':  1       low
    2    middle
    3      high
   -2       NaN
   dtype: object},
  '信用卡交易金額[新台幣]': {'type': 'numeric', 'value': [-1795429, 844247746]},
  '年月': {'type': 'datetime',
   'value': ['43',
    '48',
    '52',
    '05',
    '09',
    '13',
    '17',
    '22',
    '26',
    '31',
    '35',
    '39',
    '44',
    '18',
    '53',
    '04',
    '08',
    '12',
    '21',
    '25',
    '30',
    '34',
    '47'],
   'mapping': 1      43
   2      48
   3      52
   4      05
   5      09
   6      13
   7      17
   8      22
   9      26
   10     31
   11     35
   12     39
   13     44
   14     18
   15     53
   16     04
   17     08
   18     12
   19     21
   20     25
   21     30
   22     34
   23     47
   24    NaN
   dtype:

## Decision tree path resolver

In [ ]:
from math import log


def DecisionTreePathResolver(graph: DecisionTreeGraph, root_id: int = 0):
    paths: list[DecisionTreePath] = []

    # DFS: Depth-First Search
    def SearchPathByDFS(current_id: int = 0, path: list[int] = []):
        if not graph:
            return

        path.append(current_id)

        edge_values = list(map(lambda edge: DecisionTreeEdge(
            **edge), list(graph.edges.values())))
        outgoing_edges = list(
            filter(lambda edge: edge.head == current_id, edge_values))

        # 如果目前節點沒有出邊（即為最底層節點），將路徑加入結果中
        if len(outgoing_edges) == 0:
            last_id = path[len(path) - 1]
            last_node = DecisionTreeNode(**(graph.nodes[last_id]))
            node_labels: dict[int, list[str]] = {}

            # ! 排除 entropy 高的 path
            entropy = float(last_node.labels[0].split(" ")[2])

            if entropy > log(len(feature_names), 2) / 2:
                path.pop()
                return
            # ! ####################

            node_labels[last_id] = last_node.labels

            for i in range(0, len(path) - 1):
                node_id = path[i]
                labels = DecisionTreeNode(**(graph.nodes[node_id])).labels

                # 如果下一個的 node id 是上一個 +1 則是 true，不然的話是 false
                # * left edge <= => true
                # * right edge > => false

                next_id = path[i + 1]

                if node_id + 1 != next_id:
                    new_labels = [*labels]
                    condition = new_labels[0]
                    split_condition = condition.split(" ")
                    split_condition[1] = ">"
                    new_labels[0] = " ".join(split_condition)
                    node_labels[node_id] = new_labels
                    continue

                node_labels[node_id] = [*labels]

            paths.append(DecisionTreePath([*path], node_labels))

        # 遍歷目前節點的所有出邊
        else:
            for edge in outgoing_edges:
                next_id = edge.tail

                # 遞迴呼叫深度優先搜索
                SearchPathByDFS(next_id, path)

        # 回溯，從路徑中移除目前節點
        path.pop()

    SearchPathByDFS(root_id)

    return paths


In [ ]:
decision_tree_graph = DecisionTreeGraph(nodes, edges)
paths = DecisionTreePathResolver(decision_tree_graph, 0)
print("Path counts = {}".format(len(paths)))


Path counts = 755


## Decision tree path analyzer

In [ ]:
from math import ceil, floor


def DecisionTreePathAnalyzer(paths: list[DecisionTreePath], target_values: list[str], feature_names: list[str]):
    path_analysis_result: dict = {}
    for split_value in target_values:
        path_analysis_result[split_value] = []

    for path in paths:
        path_analysis_result_part = {}

        for feature_name in feature_names:
            path_analysis_result_part[feature_name] = data_information["feature_values"][feature_name]["value"].copy(
            )

        for node_id in path.path:
            labels = path.nodeLabel[node_id][0].split(" ")
            feature_name = labels[0]
            split_symbol = labels[1]
            split_value = float(labels[2])

            if node_id == path.path[len(path.path) - 1]:
                class_name = path.nodeLabel[node_id][3].split(" ")[2]

                sample_value = " ".join(
                    path.nodeLabel[node_id][2].split(" ")[2:]).split(", ")
                sample_value[0] = sample_value[0][1:]
                sample_value[len(sample_value) -
                             1] = sample_value[len(sample_value) - 1][0:-1]

                path_analysis_result_part["entropy"] = float(split_value)
                path_analysis_result_part["samples"] = list(
                    map(lambda value: int(value), sample_value))
                path_analysis_result_part["labels"] = target_values
                path_analysis_result_part["class"] = class_name
                path_analysis_result_part["target"] = target

                path_analysis_result[class_name].append(
                    path_analysis_result_part)
                break

            feature_type = data_information["feature_values"][labels[0]]["type"]
            split_situation = [split_symbol, feature_type]
            mapping: pd.Series = data_information["feature_values"][feature_name]["mapping"]

            match split_situation:
                case ["<=", "category"]:
                    index = path_analysis_result_part[feature_name].index(
                        mapping[floor(split_value)])
                    path_analysis_result_part[feature_name] = path_analysis_result_part[feature_name][0: index + 1]
                case ["<=", "datetime"]:
                    index = path_analysis_result_part[feature_name].index(
                        mapping[floor(split_value)])
                    path_analysis_result_part[feature_name] = path_analysis_result_part[feature_name][0: index + 1]
                case [">", "category"]:
                    index = path_analysis_result_part[feature_name].index(
                        mapping[ceil(split_value)])
                    path_analysis_result_part[feature_name] = path_analysis_result_part[feature_name][index:]
                case [">", "datetime"]:
                    index = path_analysis_result_part[feature_name].index(
                        mapping[ceil(split_value)])
                    path_analysis_result_part[feature_name] = path_analysis_result_part[feature_name][index:]
                case _:
                    print("no match case")

    return path_analysis_result


In [ ]:
path_analysis_result = DecisionTreePathAnalyzer(
    paths=paths, target_values=class_names, feature_names=feature_names)
path_analysis_result


{'high': [{'年月': ['43',
    '48',
    '52',
    '05',
    '09',
    '13',
    '17',
    '22',
    '26',
    '31',
    '35',
    '39',
    '44',
    '18',
    '53',
    '04',
    '08',
    '12',
    '21',
    '25',
    '30',
    '34',
    '47'],
   '地區': ['宜蘭縣'],
   '產業別': ['百貨'],
   '性別': ['2'],
   '教育程度類別': ['高中高職', '其他'],
   '信用卡交易筆數': ['middle'],
   'entropy': 0.49,
   'samples': [92, 0, 11],
   'labels': ['high', 'low', 'middle'],
   'class': 'high',
   'target': '信用卡交易金額[新台幣]'},
  {'年月': ['43',
    '48',
    '52',
    '05',
    '09',
    '13',
    '17',
    '22',
    '26',
    '31',
    '35',
    '39',
    '44',
    '18',
    '53',
    '04',
    '08',
    '12',
    '21',
    '25',
    '30',
    '34',
    '47'],
   '地區': ['新竹市'],
   '產業別': ['百貨'],
   '性別': ['1', '2'],
   '教育程度類別': ['高中高職', '其他'],
   '信用卡交易筆數': ['middle'],
   'entropy': 0.991,
   'samples': [80, 0, 64],
   'labels': ['high', 'low', 'middle'],
   'class': 'high',
   'target': '信用卡交易金額[新台幣]'},
  {'年月': ['43',
    '48'

## Convert path analysis result to json string

In [ ]:
json_str = json.dumps(path_analysis_result)
json_object = json.loads(json_str)

for k in json_object:
    print(k, len(json_object[k]))


high 153
low 127
middle 475


## Final result

In [ ]:
df


,年月,地區,產業別,性別,教育程度類別,信用卡交易筆數,信用卡交易金額[新台幣]
0,43,南投縣,百貨,1,高中高職,middle,middle
1,43,南投縣,百貨,1,其他,middle,low
2,43,南投縣,其他,1,博士,low,middle
3,43,南投縣,其他,1,碩士,middle,middle
4,43,南投縣,其他,1,大學,middle,high
...,...,...,...,...,...,...,...
147835,43,南投縣,文教康樂,1,其他,middle,middle
147836,43,南投縣,百貨,1,博士,low,low
147837,43,南投縣,百貨,1,碩士,middle,low
147838,43,南投縣,百貨,1,大學,middle,middle


In [ ]:
data_information


{'target_name': '信用卡交易金額[新台幣]',
 'target_values': ['high', 'low', 'middle'],
 'feature_names': ['年月', '地區', '產業別', '性別', '教育程度類別', '信用卡交易筆數'],
 'feature_values': {'信用卡交易筆數': {'type': 'category',
   'value': ['low', 'middle', 'high'],
   'mapping':  1       low
    2    middle
    3      high
   -2       NaN
   dtype: object},
  '信用卡交易金額[新台幣]': {'type': 'numeric', 'value': [-1795429, 844247746]},
  '年月': {'type': 'datetime',
   'value': ['43',
    '48',
    '52',
    '05',
    '09',
    '13',
    '17',
    '22',
    '26',
    '31',
    '35',
    '39',
    '44',
    '18',
    '53',
    '04',
    '08',
    '12',
    '21',
    '25',
    '30',
    '34',
    '47'],
   'mapping': 1      43
   2      48
   3      52
   4      05
   5      09
   6      13
   7      17
   8      22
   9      26
   10     31
   11     35
   12     39
   13     44
   14     18
   15     53
   16     04
   17     08
   18     12
   19     21
   20     25
   21     30
   22     34
   23     47
   24    NaN
   dtype:

In [ ]:
quantile_mapping


{'信用卡交易筆數': middle       (161.0, 10838.0]
 low           (-3.001, 161.0]
 high      (10838.0, 478692.0]
 dtype: interval,
 '信用卡交易金額[新台幣]': middle      (350377.28, 21600968.285]
 low         (-1795429.001, 350377.28]
 high      (21600968.285, 844247746.0]
 dtype: interval,
 '年月': 2015-11-01    (2015-10-25, 2015-11-01]
 2015-11-01    (2015-10-25, 2015-11-01]
 2015-11-01    (2015-10-25, 2015-11-01]
 2015-11-01    (2015-10-25, 2015-11-01]
 2015-11-01    (2015-10-25, 2015-11-01]
                         ...           
 2015-11-01    (2015-10-25, 2015-11-01]
 2015-11-01    (2015-10-25, 2015-11-01]
 2015-11-01    (2015-10-25, 2015-11-01]
 2015-11-01    (2015-10-25, 2015-11-01]
 2015-11-01    (2015-10-25, 2015-11-01]
 Length: 147840, dtype: interval}

## Tests

In [ ]:
import random

target_class = random.choice(class_names)
paths = path_analysis_result[target_class]
print(f"{target_class} path count: {len(paths)}")
target_path = random.choice(paths)

part_df = df.copy()

for feature in feature_names:
    value_df = pd.DataFrame(columns=df.columns)
    for value in target_path[feature]:
        value_df = pd.concat(
            [value_df, part_df.loc[(part_df[feature] == value)]], ignore_index=False)
    part_df = value_df

print(f"Target column of analysis: {target_path['target']}")
print(f"Most class of this path: {target_path['class']}")
print(
    f"Time range: {datetime_column_tuple[0][1]} to {datetime_column_tuple[0][2]}")
print(part_df[target].value_counts())

part_df


low path count: 127
Target column of analysis: 信用卡交易金額[新台幣]
Most class of this path: low
Time range: 2014-01-01 00:00:00 to 2023-02-01 00:00:00
low       97
middle    96
high       0
Name: 信用卡交易金額[新台幣], dtype: int64


,年月,地區,產業別,性別,教育程度類別,信用卡交易筆數,信用卡交易金額[新台幣]
32547,43,台東縣,百貨,1,碩士,middle,low
40611,43,台東縣,百貨,1,碩士,middle,middle
41283,48,台東縣,百貨,1,碩士,middle,middle
49347,48,台東縣,百貨,1,碩士,middle,middle
57411,48,台東縣,百貨,1,碩士,middle,middle
...,...,...,...,...,...,...,...
120579,30,台東縣,百貨,2,碩士,middle,middle
97059,34,台東縣,百貨,2,碩士,middle,low
105123,34,台東縣,百貨,2,碩士,middle,middle
99075,47,台東縣,百貨,2,碩士,middle,low
